In [128]:
import pandas as pd
import hashlib
df_business = pd.read_csv('Yelp_business.csv', sep=";")

In [129]:
df_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,['ByAppointmentOnly': 'True'],"Doctors, Traditional Chinese Medicine, Naturop...",NaN
1,qkRM_2X51Yqxk3btlwAQIg,Temple Beth-El,400 Pasadena Ave S,St. Petersburg,FL,33707,27.766590,-82.732983,3.5,5,1,NaN,"Synagogues, Religious Organizations","['Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
2,UJsufbvfyfONHeWdvAHKjA,Marshalls,21705 Village Lakes Sc Dr,Land O' Lakes,FL,34639,28.190459,-82.457380,3.5,6,1,"['RestaurantsPriceRange2': '2', 'BikeParking':...","Department Stores, Shopping, Fashion","['Monday': '9:30-21:30', 'Tuesday': '9:30-21:3..."
3,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"['Alcohol': ""'none'"", 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","['Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
4,jaxMSoInw8Poo3XeMJt8lQ,Adams Dental,15 N Missouri Ave,Clearwater,FL,33755,27.966235,-82.787412,5.0,10,1,['ByAppointmentOnly': 'True'],"General Dentistry, Dentists, Health & Medical,...","['Monday': '7:30-15:30', 'Tuesday': '7:30-15:3..."


In [130]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39221 entries, 0 to 39220
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   39221 non-null  object 
 1   name          39221 non-null  object 
 2   address       37378 non-null  object 
 3   city          39221 non-null  object 
 4   state         39221 non-null  object 
 5   postal_code   39221 non-null  int64  
 6   latitude      39221 non-null  float64
 7   longitude     39221 non-null  float64
 8   stars         39221 non-null  float64
 9   review_count  39221 non-null  int64  
 10  is_open       39221 non-null  int64  
 11  attributes    35240 non-null  object 
 12  categories    39190 non-null  object 
 13  hours         33623 non-null  object 
dtypes: float64(3), int64(3), object(8)
memory usage: 4.2+ MB


In [8]:
df_business['hours']

0                                                      NaN
1        ['Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ...
2        ['Monday': '9:30-21:30', 'Tuesday': '9:30-21:3...
3        ['Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'...
4        ['Monday': '7:30-15:30', 'Tuesday': '7:30-15:3...
                               ...                        
39216    ['Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...
39217    ['Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ...
39218    ['Monday': '0:0-0:0', 'Tuesday': '9:0-20:0', '...
39219    ['Monday': '10:0-19:0', 'Tuesday': '10:0-19:0'...
39220    ['Tuesday': '12:0-19:0', 'Wednesday': '12:0-19...
Name: hours, Length: 39221, dtype: object

In [138]:
categorias_filtrado = ['Grill', 'Pub', 'Deli', 'American', 'Mexican', 'Italian', 'Vietnamese','Ice Cream', 'Brewery', 'Sushi', 'Restaurants', 'Restaurant', 'Hotels', 'Hotel', 'Restaurante', 'Bar', 'Cafe', 'Coffe', 'Pizza', 'Bakery', 'Food', 'Diner', 'Bistro']

# Elimino primero las filas con valores NaN en la columna 'categories'
df_business_cleaned = df_business.dropna(subset=['categories'])

# Filtro el DataFrame manteniendo sólo las filas que contengan al menos una palabra de categorias_filtrado
df_business_cleaned = df_business_cleaned[df_business_cleaned['categories'].str.contains('|'.join(categorias_filtrado), case=False)]

In [139]:
df_business_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17570 entries, 3 to 39217
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   17570 non-null  object 
 1   name          17570 non-null  object 
 2   address       17211 non-null  object 
 3   city          17570 non-null  object 
 4   state         17570 non-null  object 
 5   postal_code   17570 non-null  int64  
 6   latitude      17570 non-null  float64
 7   longitude     17570 non-null  float64
 8   stars         17570 non-null  float64
 9   review_count  17570 non-null  int64  
 10  is_open       17570 non-null  int64  
 11  attributes    16791 non-null  object 
 12  categories    17570 non-null  object 
 13  hours         15118 non-null  object 
dtypes: float64(3), int64(3), object(8)
memory usage: 2.0+ MB


- 1º Genero un nuevo DataFrame Yelp Shops de acuerdo al diagrama de Entidad Relación<br>
Trabajo con los datos recopilados desde Yelp en el archivo `Yelp_shops.csv`<br>
Este DataFrame se llamará df_Yelp_shops y se unirá posteriormente con la misma información proveniente de Google para generar una única tabla en el modelo.

In [141]:
# Crear el DataFrame df_Yelp_shops con los campos requeridos
df_Yelp_shops = pd.DataFrame()

# Calcular el id_location como un hash code a partir de latitude y longitude
def hash_location(row):
    location_str = f"{row['latitude']},{row['longitude']}"
    return hashlib.sha256(location_str.encode()).hexdigest()

df_Yelp_shops['id_shop'] = df_business_cleaned.apply(hash_location, axis=1)

# Obtener el nombre (name) directamente del DataFrame df_business
df_Yelp_shops['name'] = df_business_cleaned['name']

# Obtener las estrellas (stars) directamente del DataFrame df_business
df_Yelp_shops['stars'] = df_business_cleaned['stars']

# Genero un nuevo campo 'source' que identifica de dónde proviene la información: 1: Yelp, 2: Google
df_Yelp_shops['source'] = 1

# Obtener el id_shop a partir del business_id
df_Yelp_shops['id_yelp'] = df_business_cleaned['business_id']

# Mostrar el DataFrame df_Yelp_shops resultante
df_Yelp_shops.head()


,id_shop,name,stars,source,id_yelp
3,4827cf3636be89140681c5f90fbffaba269355c875f247...,Vietnamese Food Truck,4.0,1,eEOYSgkmpB90uNA7lDOMRA
5,426f4cb6d68060ba49eeb342b4b67d95b2c831e547429e...,Zio's Italian Market,4.5,1,0bPLkL0QhhPO5kt1_EXmNQ
6,8c215e5cfca7bb82f8f09a0493b7a357b30b8786a673cb...,Romano's Macaroni Grill,2.5,1,9OG5YkX1g2GReZM0AskizA
10,9211bf207c20957ed0cd0de277f438198fa6c75cfa8452...,Charlie's Market,3.0,1,0qNpTGTcqPwOLi2hADx4Xw
11,4df4dbba86bed9df3294df40fc32b02959073c52114ed6...,Roman Forum,4.0,1,uI9XODGY_2_ieTE6xJ0myw


In [166]:
df_Yelp_shops.rename(columns={'id_yelp': 'id_source'}, inplace=True)

In [167]:
df_Yelp_shops.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17570 entries, 3 to 39217
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id_shop    17570 non-null  object 
 1   name       17570 non-null  object 
 2   stars      17570 non-null  float64
 3   source     17570 non-null  int64  
 4   id_source  17570 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 823.6+ KB


In [168]:
# Guardo el nuevo dataframe como '.csv'
df_Yelp_shops.to_csv('c:/Users/PC/Dropbox/HENRY/DATA-FT-12/Lab 03 - PF/DiagramaER/Yelp_shops.csv', index = False)

- 2º Genero el df_Ubicacion_Yelp_shops

In [82]:
# Crear el DataFrame df_Ubicacion_Yelp_shops con los campos requeridos
df_Ubicacion_Yelp_shops = pd.DataFrame()

# # Calcular el id_location como un hash code a partir de latitude y longitude
# def hash_location(row):
#     location_str = f"{row['latitude']},{row['longitude']}"
#     return hashlib.sha256(location_str.encode()).hexdigest()

df_Ubicacion_Yelp_shops['id_shop'] = df_business_cleaned.apply(hash_location, axis=1)

# Obtener el state directamente del DataFrame df_business
df_Ubicacion_Yelp_shops['state'] = df_business_cleaned['state']

# Obtener latitude, longitude, postal_code, city, y address directamente del DataFrame df_business
df_Ubicacion_Yelp_shops['latitude'] = df_business_cleaned['latitude']
df_Ubicacion_Yelp_shops['longitude'] = df_business_cleaned['longitude']
df_Ubicacion_Yelp_shops['postal_code'] = df_business_cleaned['postal_code']
df_Ubicacion_Yelp_shops['city'] = df_business_cleaned['city']
df_Ubicacion_Yelp_shops['address'] = df_business_cleaned['address']

# Mostrar el DataFrame df_Ubicacion_Yelp_shops resultante
df_Ubicacion_Yelp_shops.head()


,id_shop,state,latitude,longitude,postal_code,city,address
3,4827cf3636be89140681c5f90fbffaba269355c875f247...,FL,27.955269,-82.456320,33602,Tampa Bay,NaN
5,426f4cb6d68060ba49eeb342b4b67d95b2c831e547429e...,FL,27.916116,-82.760461,33771,Largo,2575 E Bay Dr
6,8c215e5cfca7bb82f8f09a0493b7a357b30b8786a673cb...,NV,39.476117,-119.789339,89502,Reno,5505 S Virginia St
10,9211bf207c20957ed0cd0de277f438198fa6c75cfa8452...,FL,28.010360,-82.430042,33610,Tampa,2815 E Sligh Ave
11,4df4dbba86bed9df3294df40fc32b02959073c52114ed6...,FL,28.046203,-82.505053,33618,Tampa,10440 N Dale Mabry Hwy


In [83]:
df_Ubicacion_Yelp_shops.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17235 entries, 3 to 39217
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id_shop      17235 non-null  object 
 1   state        17235 non-null  object 
 2   latitude     17235 non-null  float64
 3   longitude    17235 non-null  float64
 4   postal_code  17235 non-null  int64  
 5   city         17235 non-null  object 
 6   address      16895 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 1.1+ MB


In [95]:
# Guardo el nuevo dataframe como '.csv'
df_Ubicacion_Yelp_shops.to_csv('c:/Users/PC/Dropbox/HENRY/DATA-FT-12/Lab 03 - PF/DiagramaER/Ubicacion_Yelp_shops.csv', index = False)

- 3º Genero el df_Yelp_categories

In [145]:
# categorias_filtrado = ['Hotel', 'Hotels', 'Restaurant', 'Restaurante','Restaurants','Bar', 'Cafe', 'Coffe', 'Pizza', 'Bakery', 'Food', 'Diner', 'Bistro']
categorias_filtrado = ['Grill', 'Pub', 'Deli', 'American', 'Mexican', 'Italian', 'Vietnamese','Ice Cream', 'Brewery', 'Sushi', 'Restaurants', 'Restaurant', 'Hotels', 'Hotel', 'Restaurante', 'Bar', 'Cafe', 'Coffe', 'Pizza', 'Bakery', 'Bakeries', 'Food', 'Diner', 'Bistro']

# Elimino primero las filas con valores NaN en la columna 'categories'
df_business_cleaned = df_business.dropna(subset=['categories'])

# Filtro el DataFrame manteniendo sólo las filas que contengan al menos una palabra de categorias_filtrado
df_Yelp_categories = df_business_cleaned[df_business_cleaned['categories'].str.contains('|'.join(categorias_filtrado), case=False)]


In [146]:
df_Yelp_categories.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17570 entries, 3 to 39217
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   17570 non-null  object 
 1   name          17570 non-null  object 
 2   address       17211 non-null  object 
 3   city          17570 non-null  object 
 4   state         17570 non-null  object 
 5   postal_code   17570 non-null  int64  
 6   latitude      17570 non-null  float64
 7   longitude     17570 non-null  float64
 8   stars         17570 non-null  float64
 9   review_count  17570 non-null  int64  
 10  is_open       17570 non-null  int64  
 11  attributes    16791 non-null  object 
 12  categories    17570 non-null  object 
 13  hours         15118 non-null  object 
dtypes: float64(3), int64(3), object(8)
memory usage: 2.0+ MB


In [113]:
# Divide las filas en el DataFrame según los valores de la columna 'categories'
df_category_business = df_Yelp_categories.copy()
df_category_business['categories'] = df_category_business['categories'].str.split(', ')

# Utiliza explode para crear una fila separada para cada valor en la lista de categorías
df_category_business = df_category_business.explode('categories')
df_category_business.head(10)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"['Alcohol': ""'none'"", 'OutdoorSeating': 'None'...",Vietnamese,"['Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
3,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"['Alcohol': ""'none'"", 'OutdoorSeating': 'None'...",Food,"['Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
3,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"['Alcohol': ""'none'"", 'OutdoorSeating': 'None'...",Restaurants,"['Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
3,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"['Alcohol': ""'none'"", 'OutdoorSeating': 'None'...",Food Trucks,"['Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
5,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,0,"['OutdoorSeating': 'False', 'RestaurantsGoodFo...",Food,"['Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'..."
5,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,0,"['OutdoorSeating': 'False', 'RestaurantsGoodFo...",Delis,"['Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'..."
5,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,0,"['OutdoorSeating': 'False', 'RestaurantsGoodFo...",Italian,"['Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'..."
5,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,0,"['OutdoorSeating': 'False', 'RestaurantsGoodFo...",Bakeries,"['Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'..."
5,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,0,"['OutdoorSeating': 'False', 'RestaurantsGoodFo...",Restaurants,"['Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'..."
6,9OG5YkX1g2GReZM0AskizA,Romano's Macaroni Grill,5505 S Virginia St,Reno,NV,89502,39.476117,-119.789339,2.5,339,1,"['RestaurantsGoodForGroups': 'True', 'Restaura...",Restaurants,"['Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."


In [114]:
df_category_business.drop(columns=['is_open'], inplace=True)

In [143]:
df_category_business.head(10)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,hours
3,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,"['Alcohol': ""'none'"", 'OutdoorSeating': 'None'...",Vietnamese,"['Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
3,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,"['Alcohol': ""'none'"", 'OutdoorSeating': 'None'...",Food,"['Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
3,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,"['Alcohol': ""'none'"", 'OutdoorSeating': 'None'...",Restaurants,"['Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
3,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,"['Alcohol': ""'none'"", 'OutdoorSeating': 'None'...",Food Trucks,"['Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
5,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,"['OutdoorSeating': 'False', 'RestaurantsGoodFo...",Food,"['Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'..."
5,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,"['OutdoorSeating': 'False', 'RestaurantsGoodFo...",Delis,"['Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'..."
5,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,"['OutdoorSeating': 'False', 'RestaurantsGoodFo...",Italian,"['Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'..."
5,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,"['OutdoorSeating': 'False', 'RestaurantsGoodFo...",Bakeries,"['Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'..."
5,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,"['OutdoorSeating': 'False', 'RestaurantsGoodFo...",Restaurants,"['Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'..."
6,9OG5YkX1g2GReZM0AskizA,Romano's Macaroni Grill,5505 S Virginia St,Reno,NV,89502,39.476117,-119.789339,2.5,339,"['RestaurantsGoodForGroups': 'True', 'Restaura...",Restaurants,"['Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."


In [151]:
df_category_business.info()

<class 'pandas.core.frame.DataFrame'>
Index: 82557 entries, 3 to 39217
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   82557 non-null  object 
 1   name          82557 non-null  object 
 2   address       80638 non-null  object 
 3   city          82557 non-null  object 
 4   state         82557 non-null  object 
 5   postal_code   82557 non-null  int64  
 6   latitude      82557 non-null  float64
 7   longitude     82557 non-null  float64
 8   stars         82557 non-null  float64
 9   review_count  82557 non-null  int64  
 10  attributes    79922 non-null  object 
 11  categories    82557 non-null  object 
 12  hours         74225 non-null  object 
dtypes: float64(3), int64(2), object(8)
memory usage: 8.8+ MB


In [147]:
# categories_unique = df_category_business['categories'].str.split(', ', expand=True).stack().unique()
# categories_unique_sorted = sorted(categories_unique)
categories_unique_sorted = sorted(categorias_filtrado)
# Crear un nuevo DataFrame con las categorías únicas ordenadas alfabéticamente
df_categorias = pd.DataFrame({'category': categories_unique_sorted})
# Agregar una columna 'id' como el índice incremental
df_categorias['id_category'] = df_categorias.index + 1
# Mover la columna 'id' al principio
df_categorias = df_categorias[['id_category', 'category']]
# df_unique_categories.info()
df_categorias.head(25)


,id_category,category
0,1,American
1,2,Bakeries
2,3,Bakery
3,4,Bar
4,5,Bistro
5,6,Brewery
6,7,Cafe
7,8,Coffe
8,9,Deli
9,10,Diner


In [126]:
df_categorias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_category  23 non-null     int64 
 1   category     23 non-null     object
dtypes: int64(1), object(1)
memory usage: 496.0+ bytes


In [148]:
df_categorias.to_csv('c:/Users/PC/Dropbox/HENRY/DATA-FT-12/Lab 03 - PF/DiagramaER/categorias.csv', index = False)

In [149]:
# Filtrar el DataFrame df_category_business
df_categoria_negocio = df_category_business[df_category_business['categories'].str.contains('|'.join(categorias_filtrado), case=False)]

# Reiniciar el índice si es necesario
df_categoria_negocio.reset_index(drop=True, inplace=True)
df_categoria_negocio.head(20)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,hours
0,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,"['Alcohol': ""'none'"", 'OutdoorSeating': 'None'...",Vietnamese,"['Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
1,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,"['Alcohol': ""'none'"", 'OutdoorSeating': 'None'...",Food,"['Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
2,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,"['Alcohol': ""'none'"", 'OutdoorSeating': 'None'...",Restaurants,"['Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
3,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,"['Alcohol': ""'none'"", 'OutdoorSeating': 'None'...",Food Trucks,"['Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
4,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,"['OutdoorSeating': 'False', 'RestaurantsGoodFo...",Food,"['Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'..."
5,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,"['OutdoorSeating': 'False', 'RestaurantsGoodFo...",Delis,"['Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'..."
6,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,"['OutdoorSeating': 'False', 'RestaurantsGoodFo...",Italian,"['Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'..."
7,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,"['OutdoorSeating': 'False', 'RestaurantsGoodFo...",Bakeries,"['Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'..."
8,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,"['OutdoorSeating': 'False', 'RestaurantsGoodFo...",Restaurants,"['Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'..."
9,9OG5YkX1g2GReZM0AskizA,Romano's Macaroni Grill,5505 S Virginia St,Reno,NV,89502,39.476117,-119.789339,2.5,339,"['RestaurantsGoodForGroups': 'True', 'Restaura...",Restaurants,"['Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."


In [150]:
df_categoria_negocio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45132 entries, 0 to 45131
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   45132 non-null  object 
 1   name          45132 non-null  object 
 2   address       44254 non-null  object 
 3   city          45132 non-null  object 
 4   state         45132 non-null  object 
 5   postal_code   45132 non-null  int64  
 6   latitude      45132 non-null  float64
 7   longitude     45132 non-null  float64
 8   stars         45132 non-null  float64
 9   review_count  45132 non-null  int64  
 10  attributes    43953 non-null  object 
 11  categories    45132 non-null  object 
 12  hours         40087 non-null  object 
dtypes: float64(3), int64(2), object(8)
memory usage: 4.5+ MB


In [153]:
df_categoria_negocio['id_shop'] = df_categoria_negocio.apply(hash_location, axis=1)


C:\Users\PC\AppData\Local\Temp\ipykernel_19724\2710848754.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_categoria_negocio['id_shop'] = df_categoria_negocio.apply(hash_location, axis=1)


In [154]:
# Realiza una fusión (merge) en función de la columna 'category'
df_categoria_negocio = df_categoria_negocio.merge(df_categorias, left_on='categories', right_on='category', how='left')

# Elimina la columna 'category' que ya no es necesaria
df_categoria_negocio.drop(columns=['category'], inplace=True)

In [156]:
df_categoria_negocio.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,hours,id_shop,id_category
0,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,"['Alcohol': ""'none'"", 'OutdoorSeating': 'None'...",Vietnamese,"['Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'...",4827cf3636be89140681c5f90fbffaba269355c875f247...,24.0
1,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,"['Alcohol': ""'none'"", 'OutdoorSeating': 'None'...",Food,"['Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'...",4827cf3636be89140681c5f90fbffaba269355c875f247...,11.0
2,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,"['Alcohol': ""'none'"", 'OutdoorSeating': 'None'...",Restaurants,"['Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'...",4827cf3636be89140681c5f90fbffaba269355c875f247...,22.0
3,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,"['Alcohol': ""'none'"", 'OutdoorSeating': 'None'...",Food Trucks,"['Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'...",4827cf3636be89140681c5f90fbffaba269355c875f247...,NaN
4,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,"['OutdoorSeating': 'False', 'RestaurantsGoodFo...",Food,"['Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'...",426f4cb6d68060ba49eeb342b4b67d95b2c831e547429e...,11.0


In [158]:
columns_to_drop = ['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'attributes', 'categories', 'hours']
df_categoria_negocio.drop(columns=columns_to_drop, inplace=True)
df_categoria_negocio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45132 entries, 0 to 45131
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   review_count  45132 non-null  int64  
 1   id_shop       45132 non-null  object 
 2   id_category   23337 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.0+ MB


In [163]:
# Eliminar filas con NaN en la columna "id_category"
df_categoria_negocio.dropna(subset=['id_category'], inplace=True)

# Convertir la columna "id_category" a tipo int64
df_categoria_negocio['id_category'] = df_categoria_negocio['id_category'].astype('int64')

df_categoria_negocio.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23337 entries, 0 to 45130
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_shop      23337 non-null  object
 1   id_category  23337 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 547.0+ KB


In [164]:
df_categoria_negocio.head(20)

,id_shop,id_category
0,4827cf3636be89140681c5f90fbffaba269355c875f247...,24
1,4827cf3636be89140681c5f90fbffaba269355c875f247...,11
2,4827cf3636be89140681c5f90fbffaba269355c875f247...,22
4,426f4cb6d68060ba49eeb342b4b67d95b2c831e547429e...,11
6,426f4cb6d68060ba49eeb342b4b67d95b2c831e547429e...,16
7,426f4cb6d68060ba49eeb342b4b67d95b2c831e547429e...,2
8,426f4cb6d68060ba49eeb342b4b67d95b2c831e547429e...,22
9,8c215e5cfca7bb82f8f09a0493b7a357b30b8786a673cb...,22
10,8c215e5cfca7bb82f8f09a0493b7a357b30b8786a673cb...,16
11,9211bf207c20957ed0cd0de277f438198fa6c75cfa8452...,11


In [165]:
df_categoria_negocio.to_csv('c:/Users/PC/Dropbox/HENRY/DATA-FT-12/Lab 03 - PF/DiagramaER/categoria_negocio_Yelp.csv', index = False)

4º Genero el df_dia_horarios

In [100]:
df_Yelp_categories.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
3,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"['Alcohol': ""'none'"", 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","['Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'..."
5,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,0,"['OutdoorSeating': 'False', 'RestaurantsGoodFo...","Food, Delis, Italian, Bakeries, Restaurants","['Monday': '10:0-18:0', 'Tuesday': '10:0-20:0'..."
6,9OG5YkX1g2GReZM0AskizA,Romano's Macaroni Grill,5505 S Virginia St,Reno,NV,89502,39.476117,-119.789339,2.5,339,1,"['RestaurantsGoodForGroups': 'True', 'Restaura...","Restaurants, Italian","['Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."
10,0qNpTGTcqPwOLi2hADx4Xw,Charlie's Market,2815 E Sligh Ave,Tampa,FL,33610,28.010360,-82.430042,3.0,9,1,"['BusinessParking': ['garage': False, 'street'...","Food, Grocery, Convenience Stores",NaN
11,uI9XODGY_2_ieTE6xJ0myw,Roman Forum,10440 N Dale Mabry Hwy,Tampa,FL,33618,28.046203,-82.505053,4.0,23,0,"['BusinessParking': ['garage': False, 'street'...","Restaurants, American (New), Italian","['Monday': '11:30-21:0', 'Tuesday': '11:30-21:..."


In [103]:
categories_unique = df_Yelp_categories['categories'].str.split(', ', expand=True).stack().unique()
categories_unique_sorted = sorted(categories_unique, reverse=True)

# Crear un nuevo DataFrame con las categorías únicas ordenadas alfabéticamente
df_unique_categories = pd.DataFrame({'unique_categories': categories_unique_sorted})

df_unique_categories.head(20)

,unique_categories
0,Zoos
1,Yoga
2,Yelp Events
3,Wraps
4,Women's Clothing
5,Wineries
6,Wine Tours
7,Wine Tasting Room
8,Wine Tasting Classes
9,Wine Bars


In [55]:
days = []
open_times = []
close_times = []
business_ids = []

for idx, row in df_Yelp_categories.iterrows():
    business_id = row['business_id']
    entry = row['hours']

    if isinstance(entry, str):  # Verificar si entry es una cadena
        entry = entry.replace('[', '').replace(']', '').replace("'", "") # Eliminar caracteres no deseados
        entries = entry.split(', ')
        for e in entries:
            day, time_range = e.split(': ')
            open_time, close_time = time_range.split('-')
            business_ids.append(business_id)
            days.append(day)
            open_times.append(open_time)
            close_times.append(close_time)

df_dia_horarios_Yelp = pd.DataFrame({
    'id_yelp': business_ids,
    'day_name': days,
    'open_time': open_times,
    'close_time': close_times
})

In [56]:
df_dia_horarios_Yelp.head(10)

,id_yelp,day_name,open_time,close_time
0,eEOYSgkmpB90uNA7lDOMRA,Monday,11:0,14:0
1,eEOYSgkmpB90uNA7lDOMRA,Tuesday,11:0,14:0
2,eEOYSgkmpB90uNA7lDOMRA,Wednesday,11:0,14:0
3,eEOYSgkmpB90uNA7lDOMRA,Thursday,11:0,14:0
4,eEOYSgkmpB90uNA7lDOMRA,Friday,11:0,14:0
5,eEOYSgkmpB90uNA7lDOMRA,Saturday,5:0,10:0
6,eEOYSgkmpB90uNA7lDOMRA,Sunday,15:0,18:0
7,0bPLkL0QhhPO5kt1_EXmNQ,Monday,10:0,18:0
8,0bPLkL0QhhPO5kt1_EXmNQ,Tuesday,10:0,20:0
9,0bPLkL0QhhPO5kt1_EXmNQ,Wednesday,10:0,20:0


In [88]:
# Función para convertir a formato AM/PM
def convert_to_am_pm(time_str):
    time_obj = datetime.strptime(time_str, '%H:%M')
    am_pm_time = time_obj.strftime('%I:%M %p')
    return am_pm_time

# Aplicar la transformación a los campos open_time y close_time
df_dia_horarios_Yelp['open_time'] = df_dia_horarios_Yelp['open_time'].apply(convert_to_am_pm)
df_dia_horarios_Yelp['close_time'] = df_dia_horarios_Yelp['close_time'].apply(convert_to_am_pm)

# Imprimir el DataFrame con los campos transformados
df_dia_horarios_Yelp.head()


,id_yelp,day_name,open_time,close_time,id_shop,open_time_AMPM,close_time_AMPM
0,eEOYSgkmpB90uNA7lDOMRA,Monday,11:00,14:00,4827cf3636be89140681c5f90fbffaba269355c875f247...,11:00 AM,02:00 PM
1,eEOYSgkmpB90uNA7lDOMRA,Tuesday,11:00,14:00,4827cf3636be89140681c5f90fbffaba269355c875f247...,11:00 AM,02:00 PM
2,eEOYSgkmpB90uNA7lDOMRA,Wednesday,11:00,14:00,4827cf3636be89140681c5f90fbffaba269355c875f247...,11:00 AM,02:00 PM
3,eEOYSgkmpB90uNA7lDOMRA,Thursday,11:00,14:00,4827cf3636be89140681c5f90fbffaba269355c875f247...,11:00 AM,02:00 PM
4,eEOYSgkmpB90uNA7lDOMRA,Friday,11:00,14:00,4827cf3636be89140681c5f90fbffaba269355c875f247...,11:00 AM,02:00 PM


In [92]:
df_dia_horarios_Yelp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97231 entries, 0 to 97230
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_yelp     97231 non-null  object
 1   day_name    97231 non-null  object
 2   open_time   97231 non-null  object
 3   close_time  97231 non-null  object
 4   id_shop     97231 non-null  object
dtypes: object(5)
memory usage: 3.7+ MB


In [91]:
# Merge para agregar latitude y longitude a df_dia_horarios utilizando yelp_id como clave
# df_dia_horarios_Yelp = df_dia_horarios_Yelp.merge(df_Yelp_categories[['business_id', 'latitude', 'longitude']], left_on='id_yelp', right_on='business_id', how='left')

# Elimina la columna 'business_id' duplicada después de la fusión
df_dia_horarios_Yelp.drop(columns=['open_time_AMPM'], inplace=True)
df_dia_horarios_Yelp.drop(columns=['close_time_AMPM'], inplace=True)
df_dia_horarios_Yelp.head()


,id_yelp,day_name,open_time,close_time,id_shop
0,eEOYSgkmpB90uNA7lDOMRA,Monday,11:00,14:00,4827cf3636be89140681c5f90fbffaba269355c875f247...
1,eEOYSgkmpB90uNA7lDOMRA,Tuesday,11:00,14:00,4827cf3636be89140681c5f90fbffaba269355c875f247...
2,eEOYSgkmpB90uNA7lDOMRA,Wednesday,11:00,14:00,4827cf3636be89140681c5f90fbffaba269355c875f247...
3,eEOYSgkmpB90uNA7lDOMRA,Thursday,11:00,14:00,4827cf3636be89140681c5f90fbffaba269355c875f247...
4,eEOYSgkmpB90uNA7lDOMRA,Friday,11:00,14:00,4827cf3636be89140681c5f90fbffaba269355c875f247...


In [58]:
df_dia_horarios_Yelp['id_shop'] = df_dia_horarios_Yelp.apply(hash_location, axis=1)

In [59]:
df_dia_horarios_Yelp.head(15)

,id_yelp,day_name,open_time,close_time,latitude,longitude,id_shop
0,eEOYSgkmpB90uNA7lDOMRA,Monday,11:0,14:0,27.955269,-82.456320,4827cf3636be89140681c5f90fbffaba269355c875f247...
1,eEOYSgkmpB90uNA7lDOMRA,Tuesday,11:0,14:0,27.955269,-82.456320,4827cf3636be89140681c5f90fbffaba269355c875f247...
2,eEOYSgkmpB90uNA7lDOMRA,Wednesday,11:0,14:0,27.955269,-82.456320,4827cf3636be89140681c5f90fbffaba269355c875f247...
3,eEOYSgkmpB90uNA7lDOMRA,Thursday,11:0,14:0,27.955269,-82.456320,4827cf3636be89140681c5f90fbffaba269355c875f247...
4,eEOYSgkmpB90uNA7lDOMRA,Friday,11:0,14:0,27.955269,-82.456320,4827cf3636be89140681c5f90fbffaba269355c875f247...
5,eEOYSgkmpB90uNA7lDOMRA,Saturday,5:0,10:0,27.955269,-82.456320,4827cf3636be89140681c5f90fbffaba269355c875f247...
6,eEOYSgkmpB90uNA7lDOMRA,Sunday,15:0,18:0,27.955269,-82.456320,4827cf3636be89140681c5f90fbffaba269355c875f247...
7,0bPLkL0QhhPO5kt1_EXmNQ,Monday,10:0,18:0,27.916116,-82.760461,426f4cb6d68060ba49eeb342b4b67d95b2c831e547429e...
8,0bPLkL0QhhPO5kt1_EXmNQ,Tuesday,10:0,20:0,27.916116,-82.760461,426f4cb6d68060ba49eeb342b4b67d95b2c831e547429e...
9,0bPLkL0QhhPO5kt1_EXmNQ,Wednesday,10:0,20:0,27.916116,-82.760461,426f4cb6d68060ba49eeb342b4b67d95b2c831e547429e...


In [60]:
# Elimina las columnas 'latitude' y 'longitude' del DataFrame
df_dia_horarios_Yelp = df_dia_horarios_Yelp.drop(columns=['latitude', 'longitude'])
df_dia_horarios_Yelp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97231 entries, 0 to 97230
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id_yelp     97231 non-null  object
 1   day_name    97231 non-null  object
 2   open_time   97231 non-null  object
 3   close_time  97231 non-null  object
 4   id_shop     97231 non-null  object
dtypes: object(5)
memory usage: 3.7+ MB


In [61]:
# Función para formatear las horas
def format_hour(hour):
    if pd.notna(hour):
        hour_parts = hour.split(':')  # Divide la cadena en partes
        formatted_hour = f'{hour_parts[0]:0>2}:{hour_parts[1]:0<2}'  # Formato 'XX:00'
        return formatted_hour
    return None

# Aplica la función a las columnas 'open_time' y 'close_time'
df_dia_horarios_Yelp['open_time'] = df_dia_horarios_Yelp['open_time'].apply(format_hour)
df_dia_horarios_Yelp['close_time'] = df_dia_horarios_Yelp['close_time'].apply(format_hour)

In [93]:
df_dia_horarios_Yelp.head(15)

,id_yelp,day_name,open_time,close_time,id_shop
0,eEOYSgkmpB90uNA7lDOMRA,Monday,11:00,14:00,4827cf3636be89140681c5f90fbffaba269355c875f247...
1,eEOYSgkmpB90uNA7lDOMRA,Tuesday,11:00,14:00,4827cf3636be89140681c5f90fbffaba269355c875f247...
2,eEOYSgkmpB90uNA7lDOMRA,Wednesday,11:00,14:00,4827cf3636be89140681c5f90fbffaba269355c875f247...
3,eEOYSgkmpB90uNA7lDOMRA,Thursday,11:00,14:00,4827cf3636be89140681c5f90fbffaba269355c875f247...
4,eEOYSgkmpB90uNA7lDOMRA,Friday,11:00,14:00,4827cf3636be89140681c5f90fbffaba269355c875f247...
5,eEOYSgkmpB90uNA7lDOMRA,Saturday,05:00,10:00,4827cf3636be89140681c5f90fbffaba269355c875f247...
6,eEOYSgkmpB90uNA7lDOMRA,Sunday,15:00,18:00,4827cf3636be89140681c5f90fbffaba269355c875f247...
7,0bPLkL0QhhPO5kt1_EXmNQ,Monday,10:00,18:00,426f4cb6d68060ba49eeb342b4b67d95b2c831e547429e...
8,0bPLkL0QhhPO5kt1_EXmNQ,Tuesday,10:00,20:00,426f4cb6d68060ba49eeb342b4b67d95b2c831e547429e...
9,0bPLkL0QhhPO5kt1_EXmNQ,Wednesday,10:00,20:00,426f4cb6d68060ba49eeb342b4b67d95b2c831e547429e...


In [94]:
# Guardo el nuevo dataframe como '.csv'
df_dia_horarios_Yelp.to_csv('c:/Users/PC/Dropbox/HENRY/DATA-FT-12/Lab 03 - PF/DiagramaER/dia_horarios_Yelp.csv', index = False)

In [93]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39247 entries, 0 to 39246
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   39247 non-null  object 
 1   name          39247 non-null  object 
 2   address       37906 non-null  object 
 3   city          39247 non-null  object 
 4   state         39247 non-null  object 
 5   postal_code   39230 non-null  object 
 6   latitude      39247 non-null  float64
 7   longitude     39247 non-null  float64
 8   stars         39247 non-null  float64
 9   review_count  39247 non-null  int64  
 10  is_open       39247 non-null  int64  
 11  attributes    35611 non-null  object 
 12  categories    39224 non-null  object 
 13  hours         33103 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 4.2+ MB


**Busco qué Hoteles de la cadena Marriot está en el dataframe**

In [141]:
hyatt_rows = df_business[df_business['name'].str.contains('^Hyatt', case=False)]
hyatt_rows

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,distance
5368,_d5OAxaxRPsivgYbBq7U1A,Hyatt Place Philadelphia/King of Prussia,440 American Ave,King of Prussia,FL,19406,40.094458,-75.397480,3.5,62,1,"['BusinessAcceptsCreditCards': 'True', 'WiFi':...","Hotels, Event Planning & Services, Hotels & Tr...","['Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",3.492396e+06
6854,-1ueCbvIpUPi8KT95ETTKw,Hyatt Place Reno-Tahoe Airport,1790 E Plumb Ln,Reno,FL,89502,39.504421,-119.779589,4.0,167,1,"['RestaurantsPriceRange2': '2', 'WiFi': ""u'fre...","Hotels, Hotels & Travel, Event Planning & Serv...","['Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",5.421775e+05
8078,GOm9BM5J3FtON5_WfMhPZQ,Hyatt Place Mt. Laurel,8000 Crawford Pl,Mt. Laurel,FL,8054,39.915339,-74.945674,3.5,55,1,"['WiFi': ""u'free'"", 'RestaurantsPriceRange2': ...","Hotels & Travel, Hotels, Event Planning & Serv...",NaN,3.532869e+06
8637,BvGVVQrmZdlaPoWPxI5MYg,Hyatt Place Tucson-Central,1375 W Grant Rd,Tucson,FL,85745,32.249506,-110.994575,3.0,14,1,NaN,"Event Planning & Services, Hotels & Travel, Ho...",NaN,5.871501e+05
12125,shtNOsRUiZtTa36RNzI32A,Hyatt Place Santa Barbara,4111 State St,Santa Barbara,FL,93110,34.440279,-119.760044,3.0,67,1,"['WiFi': ""u'free'"", 'BusinessAcceptsCreditCard...","Event Planning & Services, Hotels, Hotels & Tr...","['Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",4.522561e+05
13756,MUulT4nQqWBKuq0ZX2UZtg,Hyatt House New Orleans/Downtown,1250 Poydras St,New Orleans,FL,70113,29.950768,-90.076757,3.5,76,1,"['DogsAllowed': 'False', 'WiFi': ""u'free'"", 'B...","Hotels & Travel, Hotels, Venues & Event Spaces...","['Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",2.443334e+06
15354,fZuNemxoVGRN2ENWu9zvcw,Hyatt Regency New Orleans,601 Loyola Ave,New Orleans,FL,70113,29.949638,-90.077451,3.0,521,1,"['BusinessAcceptsCreditCards': 'True', 'Restau...","Hotels & Travel, Venues & Event Spaces, Hotels...","['Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",2.443322e+06
19902,EnZA2K91xBHxL7Du-hDFfA,Hyatt Place Nashville/Opryland,220 Rudy Cir,Nashville,FL,37214,36.220822,-86.696317,3.5,62,1,"['WiFi': ""u'free'"", 'RestaurantsPriceRange2': ...","Venues & Event Spaces, Hotels, Hotels & Travel...","['Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",2.559668e+06
23840,wuw1WipH9fnr6pqrROv5dQ,Hyatt House Philadelphia/King of Prussia,240 Mall Blvd,King of Prussia,FL,19406,40.092519,-75.392873,4.0,71,1,"['BusinessAcceptsCreditCards': 'True', 'Restau...","Hotels & Travel, Event Planning & Services, Ho...","['Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",3.492809e+06
26358,OvWdL6MaF76gN7mi6RDg-w,Hyatt Place Nashville Downtown,301 3rd Avenue South,Nashville,FL,37201,36.158234,-86.774179,4.0,177,1,"['RestaurantsPriceRange2': '2', 'DogsAllowed':...","Event Planning & Services, Hotels & Travel, Ho...","['Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",2.553770e+06
